# Method of Laplace
The method of Laplace utilizes several observations and numerical integration to find the initial vector orbital elements for the body of interest. After determining these vectors, the entire orbit can be numerically integrated.

### Import modules
EphemPy is used to find the vector pointing from the earth to the sun. This is crucial for conversion from geocentric observations of the asteroid to heliocentric range and velocity vectors.

In [153]:
from __future__ import division
from vpython import *
import numpy as np
import sys
sys.path.append("./util")
from ephemPy import Ephemeris as Ephemeris_BC

### Read in data file

In [154]:
# file_path = "./data/laplace_data.csv"
file_path = "/Users/BenClark/Documents/yspa/data/all_obs.csv"
observations = np.genfromtxt(file_path, delimiter=",")#, skip_header=1)

jd_list = observations[:, 0]
ra_list = observations[:, 1]
dec_list = observations[:, 2]
# mag_list = observations[:, 3]

### Convert right ascension values to degrees (if necessary)

In [155]:
for i in xrange(len(ra_list)):
    ra_list[i] = (ra_list[i] / 24) * 360

### Parameters

In [156]:
# jd = 2457585.666667
jd = 2457597.629
r_guess = 1.5
k = 0.01720209895
dt = 0.00001

### JPL Ephemeris class
This class uses the JPL Ephemeris to allow for the calculation of the Earth-Sun vector at any JD.

In [157]:
class Ephemeris(Ephemeris_BC):

    def __init__(self, *args, **kwargs):
        Ephemeris_BC.__init__(self, *args, **kwargs)
        self.AUFAC = 1.0/self.constants.AU
        self.EMFAC = 1.0/(1.0+self.constants.EMRAT)

    def position(self, t, target, center):
        pos = self._position(t, target)
        if center != self.SS_BARY:
            pos = pos - self._position(t, center)
        return pos
    
    def _position(self, t, target):
        if target == self.SS_BARY:
            return numpy.zeros((3,), numpy.float64)
        if target == self.EM_BARY:
            return Ephemeris_BC.position(self, t, self.EARTH)*self.AUFAC
        pos = Ephemeris_BC.position(self, t, target)*self.AUFAC
        if target == self.EARTH:
            mpos = Ephemeris_BC.position(self, t, self.MOON)*self.AUFAC
            pos = pos - mpos*self.EMFAC
        elif target == self.MOON:
            epos = Ephemeris_BC.position(self, t, self.EARTH)*self.AUFAC
            pos = pos + epos - pos*self.EMFAC
        return pos

### Numerical differentiation functions
These two functions differentiate with respect to tau, modified days. This allows for a simplification of the units, because the gravitational constant does not need to be taken into account when considering the acceleration due to gravity.

In [158]:
def indexed_d_by_d_tau(x_list, y_list, i, f):
    x0 = x_list[i]
    xf = x_list[f]

    y0 = y_list[i]
    yf = y_list[f]

    return d_by_d_tau(x0, xf, y0, yf)

def d_by_d_tau(x0, xf, y0, yf):
    delta_y = yf - y0
    delta_x = (xf - x0) * k

    slope = delta_y / delta_x
    avg_x = (x0 + xf) / 2

    return (avg_x, slope)

### Rho hats function
Because our observations have only the right ascension and declination of the asteroid at particular times, the Earth-asteroid unit vector must be calculated for each JD.

In [159]:
def get_rho_hats(alpha_list, delta_list):
    rho_list = []
    
    for i in range(len(alpha_list)):
        alpha_i = radians(alpha_list[i])
        delta_i = radians(delta_list[i])

        rho_i = vector(np.cos(alpha_i) * np.cos(delta_i), \
                          np.sin(alpha_i) * np.cos(delta_i), \
                          np.sin(delta_i))

        rho_list.append(rho_i)
        
    rho_list = np.array(rho_list)

    return rho_list

### Get first derivatives
This function will take in an array of vectors and their corresponding times and output the derivatives on each day.

In [160]:
def get_dots(rho_hats_list, jd_list):
    rho_dot_list = []
    rho_dot_times = []
    final_f = len(jd_list) - 1

    last_day = jd_list[0]
    pos = 0
    i = 0
    f = 0
    for day in jd_list:
        if np.floor(day) != np.floor(last_day):
            f = pos - 1
            if i == f:
                raise ValueError("Each day must have multiple observations")
            if f != -1:
                time_i, rho_dot_i = indexed_d_by_d_tau(jd_list, rho_hats_list, i, f)
                rho_dot_list.append(rho_dot_i)
                rho_dot_times.append(time_i)
            i = pos
        if pos == final_f:
            time_i, rho_dot_i = indexed_d_by_d_tau(jd_list, rho_hats_list, i, final_f)
            rho_dot_list.append(rho_dot_i)
            rho_dot_times.append(time_i)
        pos += 1
        last_day = day

    return (rho_dot_times, rho_dot_list)

### Get second derivative
Given a list of first derivatives (one per day day) and a target day, this function will calculate the derivative around the target day.

In [161]:
def get_dot_dot(rho_hat_dot_list, jd_list, jd):
    floored_jd_list = []
    for day in jd_list:
        floored_jd_list.append(np.floor(day))

    jd_index = floored_jd_list.index(np.floor(jd))
    i = jd_index - 1
    f = jd_index + 1

    time, rho_hat_dot_dot = indexed_d_by_d_tau(jd_list, rho_hat_dot_list, i, f)

    return (time, rho_hat_dot_dot)

### Convert back to celestial coordinates
The opposite of the function to get rho hats, this takes in a rho hat and outputs the right ascension and declination.

In [162]:
def get_celestial_coordinates(rho_hat):
    dec = np.arcsin(rho_hat.z)
    
    ra = np.arccos(rho_hat.x / (np.cos(dec)))

    if rho_hat.y < 0:
        ra =  2 * np.pi - ra

    return np.degrees(ra), np.degrees(dec)

### Error analysis functions
These two functions help determine the root mean square to determine how far the model deviates from the observations.

In [163]:
def get_chi_square(observation_list, expectation_list):
    chi_square = 0

    for i in range(len(observation_list)):
        chi_square += (observation_list[i] - expectation_list[i]) ** 2

    return chi_square

def get_rms(chi_square, dof):
    return np.sqrt((chi_square) / (dof - 3))

## Method of Laplace
This function calls on many of the functions above to take in a list of observations and output the vector orbital elements of the preliminary orbit model. It relies heavily on numerical integration, which leads the method to fail in certain instances.

Although the `r` vector can be calculated numerically, it is common very common to just solve for it through iteration. In this method, an "r guess" is inputted, which acts as a seed upon which the method will build to converge on an `r`. The method returns this r along with a velocity vector `r_dot`.

In [164]:
def laplace(alpha_list, delta_list, jd_list, jd, r_guess):
    """Determines preliminary position and acceleration vectors
    
    Given list of RA and Dec values for the asteroid, determines
    preliminary guesses for the r (position) and rdot(velocity)
    vectors for the asteroid's orbit using the Method of Laplace.

    Args:
        alpha_list: A list of alpha (Right Ascention) values observed
            for the asteroid
        delta_list: A list of correspoinding delta observatios of the
            asteroid.
        jd_list: A list of corresponding Julian day numbers for the
            observations
        jd: A target Julian date around which the preliminary vector
            orbital elements are calculated.
        r_guess: A guess for the r vector, which is used as a seed
            to iteratively solve for the converging r vector.

    Returns:
        r_vector: A vpython vector of an initial position. It is the
            radial vector pointing from the sun to the asteroid.
        r_dot_vector: A vpython vector of initial position. It points tangent
            to the asteroid's elliptical path.

    Raises:
        ValueError: All three inputted lists must be have the same
        length. The target JD must be in the JD observation list.
    """

    if len(alpha_list) != len(delta_list) \
            or len(jd_list) != len(alpha_list) \
            or len(delta_list) != len(jd_list):
        raise ValueError("Lists must be the same length")

    if not (jd in jd_list):
        raise ValueError("Julian date must be in jd_list")

    # rho-hat and its derivatives
    rho_hat_list = get_rho_hats(alpha_list, delta_list)
    rho_hat_dot_times, rho_hat_dot_list = get_dots(rho_hat_list, jd_list)
    rho_hat_dot_dot_time, rho_hat_dot_dot = get_dot_dot(rho_hat_dot_list, \
        rho_hat_dot_times, jd)


    # R and its derivatives
    ephem = Ephemeris('405')
    R = ephem.position(jd, 10, 2)

    R_list = []
    for day in jd_list:
        R_i = ephem.position(day, 10, 2)
        R_i_x = R_i[0]
        R_i_y = R_i[1]
        R_i_z = R_i[2]
        R_i = vector(R_i_x, R_i_y, R_i_z)
        R_list.append(R_i)

    R_dot_times, R_dot_list = get_dots(R_list, jd_list)

    R_dot_dot_time, R_dot_dot = get_dot_dot(R_dot_list, R_dot_times, jd)


    # Find individual particular values
    rho_hat_dot_floored_times = []
    for time in rho_hat_dot_times:
        rho_hat_dot_floored_times.append(np.floor(time))
    rho_hat_dot_floored_times = np.array(rho_hat_dot_floored_times)

    rho_hat_index = -1
    rho_hat_dot_index = -1
    if jd in jd_list:
        rho_hat_index = jd_list.tolist().index(jd)
    if np.floor(jd) in rho_hat_dot_floored_times:
        rho_hat_dot_index = rho_hat_dot_floored_times.tolist().index(np.floor(jd))

    # final values
    rho_hat = rho_hat_list[rho_hat_index]
    rho_hat_dot = rho_hat_dot_list[rho_hat_dot_index]

    R = R_list[rho_hat_index]
    R_dot = R_dot_list[rho_hat_dot_index]

    r = r_guess
    old_r = 0

    r_vector = None
    r_dot_vector = None

    # ABCD
    a = (dot(rho_hat, cross(rho_hat_dot, R_dot_dot))) / \
        (dot(rho_hat, cross(rho_hat_dot, rho_hat_dot_dot)))

    b = (dot(rho_hat, cross(rho_hat_dot, R))) / \
        (dot(rho_hat, cross(rho_hat_dot, rho_hat_dot_dot)))

    c = (dot(rho_hat, cross(R_dot_dot, rho_hat_dot_dot))) / \
        (dot(rho_hat, cross(rho_hat_dot, rho_hat_dot_dot)))

    d = (dot(rho_hat, cross(R, rho_hat_dot_dot))) / \
        (dot(rho_hat, cross(rho_hat_dot, rho_hat_dot_dot)))

    while abs(r - old_r) > 0.000000000001:
        rho = a + (b / r ** 3)
        rho_dot = c + (d / r ** 3)

        r_vector = rho * rho_hat - R
        r_dot_vector = rho * rho_hat_dot + rho_dot * rho_hat - R_dot

        old_r = r
        r = mag(r_vector)

    return r_vector, r_dot_vector

In [168]:
r_vector, r_dot_vector = laplace(ra_list, dec_list, jd_list, jd, r_guess)
print "r: ", r_vector
print "r-dot: ", r_dot_vector

r:  <0.727497, -0.928527, -0.313788>
r-dot:  <0.788670, 0.668045, 0.175360>


### Error analysis
The goodness of fit of this model can be analyzed by comparing what it predicts to what was observed. To do this, the orbit must be integrated forward and backward from the calculated initial conditions. Also, the Earth-Sun vector must be calculated on each day.

In [166]:
def get_root_mean_squares(ra_list, dec_list, jd_list, r_vector, r_dot_vector, jd):
    
    # initialize empty lists
    ra_observation_list = []
    ra_expectation_list = []

    dec_observation_list = []
    dec_expectation_list = []

    for day in jd_list:
        r = r_vector
        r_dot = r_dot_vector
        counter = 0
        
        index = jd_list.tolist().index(day)
        time = jd

        # integrate back
        if day < jd:
            while time > day:
                counter += 1
                r, r_dot = rk4(r, r_dot, grav_acceleration, -dt)
                time -= dt / k

        # integrate forward
        if day > jd:
            while time < day:
                counter += 1
                r, r_dot = rk4(r, r_dot, grav_acceleration, dt)
                time += dt / k

        # get Earth-Sun vector
        ephem = Ephemeris('405')
        R = ephem.position(day, 10, 2)
        R_x = R[0]
        R_y = R[1]
        R_z = R[2]
        R = vector(R_x, R_y, R_z)

        # calculate modeled values
        rho = R + r
        rho_hat = rho / mag(rho)
        ra, dec = get_celestial_coordinates(rho_hat)

        # get observed values
        obs_ra = ra_list[index]
        obs_dec = dec_list[index]

        # append both values to lists
        ra_observation_list.append(obs_ra)
        ra_expectation_list.append(ra)

        dec_observation_list.append(obs_dec)
        dec_expectation_list.append(dec)

    chi_square_ra = get_chi_square(ra_observation_list, ra_expectation_list)
    chi_square_dec = get_chi_square(dec_observation_list, dec_expectation_list)

    rms_ra = get_rms(chi_square_ra, len(jd_list))
    rms_dec = get_rms(chi_square_dec, len(jd_list))
    
    return (rms_ra, rms_dec)

In [167]:
rms_ra, rms_dec = get_root_mean_squares(ra_list, dec_list, jd_list, r_vector, r_dot_vector, jd)
print "Right ascension root mean square: ", rms_ra
print "Declination root mean square: ", rms_dec

Right ascension root mean square:  0.0699520544664
Declination root mean square:  0.0376076012764
